In [2]:
import pygame
import carla
import random
import time
import math
import collections
from collections import Counter
import pandas as pd
import itertools
from copy import deepcopy
import warnings
import numpy as np
import random


warnings.filterwarnings("ignore")
from utils import *

pygame 2.3.0 (SDL 2.24.2, Python 3.9.0)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
distance_to_junc = 20
direction_angle = 20
radius = 100

actor_list = []

for actor in actor_list:
    actor.destroy()
actor_list = []

client = carla.Client("localhost", 2000)
world = client.get_world()

blueprint_library = world.get_blueprint_library()
world_map = world.get_map()
spawn_points = world.get_map().get_spawn_points()
print(world.get_actors())

[Actor(id=24, type=spectator), Actor(id=25, type=traffic.stop), Actor(id=26, type=traffic.stop), Actor(id=27, type=traffic.stop), Actor(id=28, type=traffic.stop), Actor(id=29, type=traffic.stop), Actor(id=30, type=traffic.stop), Actor(id=31, type=traffic.stop), Actor(id=32, type=traffic.stop), Actor(id=33, type=traffic.unknown), Actor(id=34, type=traffic.unknown), Actor(id=35, type=traffic.yield), Actor(id=36, type=traffic.yield), Actor(id=37, type=traffic.yield), Actor(id=38, type=traffic.unknown), Actor(id=39, type=traffic.unknown), Actor(id=40, type=traffic.yield), Actor(id=41, type=traffic.speed_limit.40), Actor(id=42, type=traffic.speed_limit.40), Actor(id=43, type=traffic.speed_limit.30), Actor(id=44, type=traffic.speed_limit.30), Actor(id=45, type=traffic.speed_limit.30), Actor(id=46, type=traffic.speed_limit.30), Actor(id=47, type=traffic.speed_limit.30), Actor(id=48, type=traffic.speed_limit.30), Actor(id=49, type=traffic.speed_limit.30), Actor(id=50, type=traffic.speed_limit.

In [15]:
manual_car = world.get_actor(153)
ego_vehicle = world.get_actor(153)
#second_vehicle = world.get_actor(153)
print(ego_vehicle.get_location())

Location(x=48.692432, y=-173.736771, z=0.196287)


In [6]:
def detect_surronding_cars(
    ego_location,
    ego_vehicle,
    matrix,
    road_lane_ids,
    world,
    radius,
    on_highway,
    highway_shape,
    ghost=False,
):
    """
    Detect surrounding cars and update the city matrix accordingly.

    Parameters:
        ego_vehicle (Vehicle): The ego vehicle for which to detect surrounding cars.
        matrix (collections.OrderedDict): The city matrix representing the lanes around the ego vehicle.
        road_lanes_ids (list of str): List of road and lane IDs in the world map.
        world (carla.World): The game world where the simulation is running.
        radius (float): The radius within which to detect surrounding cars.

    Returns:
        collections.OrderedDict: The updated city matrix with detected surrounding cars.

    Description:
        This function detects surrounding cars within a specified radius from the ego vehicle and
        updates the city matrix accordingly. The city matrix is represented as an ordered dictionary
        with lane IDs as keys and lists of integers as values. Each list corresponds to the direction
        of the lane, where:
            - 0 indicates a lane in the same direction as the ego vehicle's current lane.
            - 1 indicates the ego vehicle's current lane.
            - 2 indicates a lane with a detected surrounding car.
            - 3 indicates an empty lane with no traffic.

        The function calculates the position of the detected cars in the city matrix and updates the
        corresponding cells in the matrix to indicate the presence of surrounding cars. The matrix is
        updated based on the lanes and positions/distance of the detected cars relative to the ego vehicle.

    Note:
        The city matrix should be pre-generated using the 'create_city_matrix' function.
        The returned city matrix may contain lane directions that are not within the ego vehicle's vicinity,
        as determined by the input road_lane_ids and world.
    """
    world_map = world.get_map()
    # Get all surronding cars in specified radius
    surrounding_cars = []
    surrounding_cars_on_highway_entryExit = []
    ego_vehicle_waypoint = world_map.get_waypoint(ego_location)
    ego_vehicle_road_id = ego_vehicle_waypoint.road_id

    for actor in world.get_actors():
        if "vehicle" in actor.type_id and (actor.id != ego_vehicle.id or ghost):
            distance_to_actor = distance(actor.get_location(), ego_location)
            if distance_to_actor <= radius:
                surrounding_cars.append(actor)

    _, next_lanes = check_road_change(ego_location, road_lane_ids, True, world_map)
    _, prev_lanes = check_road_change(ego_location, road_lane_ids, False, world_map)
    lanes_exist_futher = False
    lanes = []
    for id in road_lane_ids:
        if str(ego_vehicle_road_id) == id.split("_")[0]:
            lanes.append(id.split("_")[1])
    try:
        if next_lanes and matrix:
            # lanes = [road_lane.split("_")[1] for road_lane in matrix.keys()]
            lanes_exist_futher = all(lane in next_lanes for lane in lanes) or all(
                lane in lanes for lane in next_lanes
            )
    except IndexError:
        pass

    lanes_existed_before = False
    try:
        if prev_lanes and matrix:
            # lanes = [road_lane.split("_")[1] for road_lane in matrix.keys()]
            lanes_existed_before = all(lane in prev_lanes for lane in lanes) or all(
                lane in lanes for lane in prev_lanes
            )
    except IndexError:
        pass

    # in the following, ignore cars that are on highway exit/entry lanes
    if not highway_shape is None:
        entry_wps = highway_shape[2]
        exit_wps = highway_shape[3]

        entry_road_id = []
        exit_road_id = []
        entry_city_road = []
        exit_city_road = []
        entry_highway_road = []
        exit_highway_road = []
        if entry_wps:
            for entry_wp in entry_wps[0]:
                entry_city_road.append(entry_wp.previous(3)[0].road_id)
                entry_road_id.append(entry_wp.road_id)
            for entry_wp in entry_wps[1]:
                entry_highway_road.append(entry_wp.next(3)[0].road_id)
            if entry_wp.next(3)[0] and entry_wp.next(3)[0].get_left_lane() and entry_wp.next(3)[0].road_id == entry_wp.next(3)[0].get_left_lane().road_id:
                entry_highway_road = []
        if exit_wps:
            for exit_wp in exit_wps[1]:
                exit_city_road.append(exit_wp.next(3)[0].road_id)
                exit_road_id.append(exit_wp.road_id)
            for exit_wp in exit_wps[0]:
                exit_highway_road.append(exit_wp.previous(3)[0].road_id)
            if exit_wp.next(3)[0] and exit_wp.next(3)[0].get_left_lane() and exit_wp.next(3)[0].road_id == exit_wp.next(3)[0].get_left_lane().road_id:
                exit_highway_road = []
       
        
    # Update matrix based on the lane and position/distance to ego vehicle of other car
    if (
        on_highway
        and (not highway_shape is None)
        and (
            ego_vehicle_road_id
            in entry_road_id
            + entry_city_road
            + exit_road_id
            + exit_city_road
            + exit_highway_road
            + entry_highway_road
        )
    ):
        surrounding_cars_on_highway_entryExit.append(ego_vehicle)

    for car in surrounding_cars:
        # Get road and lane_id of other car
        other_car_waypoint = world_map.get_waypoint(car.get_location())
        other_car_lane_id = other_car_waypoint.lane_id
        other_car_road_id = other_car_waypoint.road_id
        other_car_road_lane_id = str(other_car_road_id) + "_" + str(other_car_lane_id)

        # ignore car on highway entry / Exit
        if (
            on_highway
            and (not highway_shape is None)
            and (
                other_car_road_id
                in entry_road_id
                + entry_city_road
                + exit_road_id
                + exit_city_road
                + exit_highway_road
                + entry_highway_road
            )
        ):
            surrounding_cars_on_highway_entryExit.append(car)
            continue
        col = calculate_position_in_matrix(
            ego_location,
            ego_vehicle,
            car,
            matrix,
            world_map,
            ego_vehicle.get_velocity(),
            ghost,
        )
        if col is None:
            continue
        if matrix:
            if other_car_road_lane_id in matrix.keys():
                if car.id == ego_vehicle.id:
                    matrix[other_car_road_lane_id][col] = 1
                else:
                    matrix[other_car_road_lane_id][col] = 2
                continue
            elif (lanes_exist_futher or lanes_existed_before) and str(
                other_car_lane_id
            ) in [str(road_lane.split("_")[1]) for road_lane in matrix.keys()]:
                if car.id == ego_vehicle.id:
                    matrix[str(ego_vehicle_road_id) + "_" + str(other_car_lane_id)][
                        col
                    ] = 1
                else:
                    matrix[str(ego_vehicle_road_id) + "_" + str(other_car_lane_id)][
                        col
                    ] = 2

    return matrix, surrounding_cars_on_highway_entryExit

In [7]:
def get_right_lane_wp(wps):
    right_lane_wp = wps[0][0]  # first wp of group
    for wp in wps:
        if abs(wp[0].lane_id) > abs(right_lane_wp.lane_id):
            right_lane_wp = wp[0]
    return right_lane_wp

In [8]:
def insert_in_matrix(matrix, car, ego_vehicle, col, row):
    if car.id == ego_vehicle.id:
        matrix[row][1][col] = 1
    else:
        if matrix[row][1][col] == 1:
            dot_product = check_car_in_front_or_behind(
                ego_vehicle.get_location(),
                car.get_location(),
                ego_vehicle.get_transform().rotation,
            )
            if dot_product > 0:
                if row == 6 or row == 7:
                    matrix[row - 1][1][col] = 2
                elif row == 5 and col < 7:
                    matrix[row][1][col + 1] = 2
            else:
                if row == 6:
                    matrix[row + 1][1][col] = 2
                elif row == 5 and col > 0:
                    matrix[row][1][col - 1] = 2
        else:
            matrix[row][1][col] = 2

In [26]:
def check_ego_on_highway(ego_vehicle_location, road_lane_ids, world_map):
    waypoints = []
    ego_waypoint = world_map.get_waypoint(ego_vehicle_location)
    waypoints.append(ego_waypoint)
    if ego_waypoint.get_left_lane():
        waypoints.append(ego_waypoint.get_left_lane())
    if len(waypoints) > 1:
        for wp in waypoints:
            ego_vehilce_road_id = wp.road_id
            lanes = []
            for id in road_lane_ids:
                if str(ego_vehilce_road_id) == id.split("_")[0]:
                    lanes.append(id.split("_")[1])
            lanes = [int(lane) for lane in lanes]
            if len(lanes) >= 6 or (
                sorted(lanes) == list(range(min(lanes), max(lanes) + 1)) and len(lanes) >= 3
            ):
                return True
            else:
                continue
        return False
    else:
        return False

In [10]:
def update_matrix(
    world_map,
    ego_vehicle,
    ego_location,
    highway_shape,
    wps,
    matrix,
    ego_waypoint,
    junction,
    cars_on_entryExit,
    ghost=False,
    on_entry = False,
):
    if highway_shape[1] == 3:
        matrix[list(matrix.keys())[5]] = [0, 0, 0, 0, 0, 0, 0, 0]

    matrix = list(matrix.items())
    exit_entry_found = False
    entry_wps, exit_wps = highway_shape[2], highway_shape[3]

    # TODO: put in a config file
    # set the angle threshold depending on curve type
    if highway_shape[0] == "entry_and_exit":
        degree = 30
    elif highway_shape[0] in ["single_entry", "single_exit"]:
        degree = 20
    elif highway_shape[0] in ["dual_entry"] or highway_shape[0] in ["dual_exit"]:
        degree = 18  # TODO: test and fine tuning of degree

    # get right_lane_end_wp & right_lane_start_wp for dual entry & exit surrounding cars angle calculation
    for wps_group in wps:
        if (len(wps_group) == 4) and (wps_group[0][1] == "end"):
            right_lane_end_wp = get_right_lane_wp(wps_group)
        elif (len(wps_group) == 4) and (wps_group[0][1] == "start"):
            right_lane_start_wp = get_right_lane_wp(wps_group)

    # First: Look behind ego
    exit_entry_found, col_entryExit, matrix = search_entry_or_exit(
        world_map,
        ego_location,
        "behind",
        exit_entry_found,
        matrix,
        highway_shape,
        wps,
        entry_wps,
        exit_wps,
        degree,
        junction.id,
        right_lane_end_wp,
        right_lane_start_wp,
    )
    # if not yet found, look in front
    if exit_entry_found == False:
        _, col_entryExit, matrix = search_entry_or_exit(
            world_map,
            ego_location,
            "before",
            exit_entry_found,
            matrix,
            highway_shape,
            wps,
            entry_wps,
            exit_wps,
            degree,
            junction.id,
            right_lane_end_wp,
            right_lane_start_wp,
        )

    # get road ids of: entry/exit, city road before and highway road after junction object
    entry_road_id = []
    exit_road_id = []
    entry_city_road = []
    exit_city_road = []
    entry_highway_road = []
    exit_highway_road = []
    entry_highway_lane_id = []
    exit_highway_lane_id = []

    if entry_wps:
        for entry_wp in entry_wps[0]:
            entry_city_road.append(entry_wp.previous(3)[0].road_id)
            entry_road_id.append(entry_wp.road_id)
        for entry_wp in entry_wps[1]:
            entry_highway_road.append(entry_wp.next(3)[0].road_id)
            entry_highway_lane_id.append(abs(entry_wp.next(3)[0].lane_id))
    if exit_wps:
        for exit_wp in exit_wps[1]:
            exit_city_road.append(exit_wp.next(3)[0].road_id)
            exit_road_id.append(exit_wp.road_id)
        for exit_wp in exit_wps[0]:
            exit_highway_road.append(exit_wp.previous(3)[0].road_id)
            exit_highway_lane_id.append(abs(exit_wp.previous(3)[0].lane_id))


    # detect surrounding cars on exit/entry
    for car in cars_on_entryExit:
        col = calculate_position_in_matrix(
            ego_location,
            ego_vehicle,
            car,
            dict(matrix),
            world_map,
            ego_vehicle.get_velocity(),
            ghost,
        )
        if col is None:
            continue
        # matrix = list(matrix)
        if matrix:
            ego_already_in_matrix = False
            for key, row in matrix:
                if 1 in row:
                    ego_already_in_matrix = True

            if (ego_already_in_matrix and car.id == ego_vehicle.id):
                continue
            other_car_waypoint = world_map.get_waypoint(car.get_location())
            other_car_road_id = other_car_waypoint.road_id
            
            if other_car_road_id in entry_city_road + exit_city_road and not on_entry:
                continue
            distance_clostest_starting_waypoint = distance(get_clostest_starting_waypoint(junction.get_waypoints(carla.LaneType().Driving), car.get_location()).transform.location, car.get_location())
            if other_car_road_id in entry_city_road + exit_city_road:
                if distance_clostest_starting_waypoint > 30:
                    continue
                elif distance_clostest_starting_waypoint < 15:
                    row = 6
                else:
                    row = 7
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    # dual entry
                    entry_road_id = [
                        x for _, x in sorted(zip(entry_highway_lane_id, entry_road_id))
                    ]
                    entry_highway_lane_id.sort()
                    if other_car_road_id == entry_road_id[0]:
                        col_entry = col_entryExit
                    elif other_car_road_id == entry_road_id[1]:
                        col_entry = col_entryExit + 1
                elif highway_shape[0] in ["dual_exit"]:
                    # dual exit
                    exit_road_id = [
                        x for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                    ]
                    exit_highway_lane_id.sort()
                    if other_car_road_id == exit_road_id[0]:
                        col_exit = col_entryExit + 1
                    elif other_car_road_id == exit_road_id[1]:
                        col_exit = col_entryExit  # + 1
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            


                if entry_city_road == exit_city_road:
                    if is_junction_behind(other_car_waypoint, 40):
                        check_junction = get_junction_behind(other_car_waypoint, 40)
                        if is_highway_junction(car, other_car_waypoint, check_junction, road_lane_ids, direction_angle):
                            on_exit_street = True
                        else:
                            on_exit_street = False
                else:
                    on_exit_street = False
                if (other_car_road_id in entry_city_road) and (col_entry < 8) and not on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)
                elif (other_car_road_id in exit_city_road) and (col_exit < 8) and on_exit_street:
                    insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)
                continue
                
            distance_junction_end = get_distance_junction_end(
                other_car_waypoint
            )
            distance_junction_start = get_distance_junction_start(
                other_car_waypoint
            )
            if other_car_road_id in entry_road_id + entry_highway_road:
                if highway_shape[0] == "dual_entry": 
                    yaw_difference = abs(
                        right_lane_end_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_end_wp.previous(distance_junction_end)[
                            0
                        ].transform.rotation.yaw
                    )
                else:
                    yaw_difference = abs(entry_wps[1][0].transform.rotation.yaw) - abs(
                        entry_wps[1][0]
                        .previous(distance_junction_end)[0]
                        .transform.rotation.yaw
                    )
            elif other_car_road_id in exit_road_id + exit_highway_road:
                if highway_shape[0] == "dual_exit":
                    yaw_difference = abs(
                        right_lane_start_wp.transform.rotation.yaw
                    ) - abs(
                        right_lane_start_wp.next(distance_junction_start)[
                            0
                        ].transform.rotation.yaw
                    )
                else:

                    yaw_difference = abs(exit_wps[0][0].transform.rotation.yaw) - abs(
                        exit_wps[0][0]
                        .next(distance_junction_start)[0]
                        .transform.rotation.yaw
                    )
             
            already_in_curve = abs(yaw_difference) > degree
            already_after_curve = abs(yaw_difference) > degree * 2
            if not on_entry:
                if already_after_curve:
                    row = 7
                elif already_in_curve:

                    row = 6
                else:
                    row = 5
            else:
                row = 5
                """
                distance_bounding_box = distance(junction.bounding_box.location, car.get_location())
                print(distance_bounding_box)
                if distance_bounding_box < 30:
                    row = 5
                if distance_bounding_box < 40:
                    row = 6
                else:
                    row = 7
                """
            if already_in_curve:
                if highway_shape[0] in ["entry_and_exit"]:
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                elif highway_shape[0] in ["dual_entry"]:
                    # dual entry
                    entry_road_id = [
                        x for _, x in sorted(zip(entry_highway_lane_id, entry_road_id))
                    ]
                    entry_highway_lane_id.sort()
                    if other_car_road_id == entry_road_id[0]:
                        col_entry = col_entryExit
                    elif other_car_road_id == entry_road_id[1]:
                        col_entry = col_entryExit + 1
                elif highway_shape[0] in ["dual_exit"]:
                    # dual exit
                    exit_road_id = [
                        x for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                    ]
                    exit_highway_lane_id.sort()
                    if other_car_road_id == exit_road_id[0]:
                        col_exit = col_entryExit + 1
                    elif other_car_road_id == exit_road_id[1]:
                        col_exit = col_entryExit  # + 1
                else:
                    col_entry = col_entryExit
                    col_exit = col_entryExit
            else:
                if (highway_shape[1] == 3) and (col <= (col_entryExit + 1)):
                    col_entry = col_entryExit + 2
                    col_exit = col_entryExit
                else:
                    if highway_shape[0] in ["dual_entry"]:
                        # dual entry
                        entry_road_id = [
                            x
                            for _, x in sorted(
                                zip(entry_highway_lane_id, entry_road_id)
                            )
                        ]
                        entry_highway_lane_id.sort()
                        if other_car_road_id == entry_road_id[0]:
                            row = row - 1
                    elif highway_shape[0] in ["dual_exit"]:
                        # dual exit
                        exit_road_id = [
                            x
                            for _, x in sorted(zip(exit_highway_lane_id, exit_road_id))
                        ]
                        exit_highway_lane_id.sort()
                        if other_car_road_id == exit_road_id[0]:
                            row = row - 1
                    col_entry = col
                col_exit = col

            entry_streets = entry_road_id + entry_highway_road
            exit_streets = exit_road_id + exit_highway_road
            if (other_car_road_id in entry_streets) and (col_entry < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_entry, row)

            elif (other_car_road_id in exit_streets) and (col_exit < 8):
                insert_in_matrix(matrix, car, ego_vehicle, col_exit, row)

    return dict(matrix)

In [54]:
same_junction = False
on_junction = False
junction_old = None
wps_old = None
highway_shape_old = None
highway_shape = None
junction_shape = None
matrix = None
junction_id = None
street_type = None 
road_lane_ids = get_all_road_lane_ids(world_map = world.get_map())
on_entry = False
junction_id_skip = None
df = initialize_dataframe()
#try:
while True:
    print("----------------------------------")
    ego_location = ego_vehicle.get_location()
    ego_waypoint = world_map.get_waypoint(ego_location)
    ego_on_highway = check_ego_on_highway(ego_location, road_lane_ids, world_map)
    # Preprocessing
    if ego_waypoint.is_junction and ego_waypoint.get_junction().id != junction_id:
        print("pre on junction")
        on_junction = True
        junction = ego_waypoint.get_junction()
        junction_id = junction.id
    elif on_junction:
        junction_old = junction
        wps_old = wps
        highway_shape_old = highway_shape
        on_junction = False
        #same_junction = False
    elif is_junction_ahead(ego_waypoint, distance_to_junc):
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        highway_junction = is_highway_junction(ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle)
        same_junction = True

    # CASES
    # 1. On city junction
    if ego_waypoint.is_junction and not highway_junction and junction.id != 459:
        street_type = "On junction"
        print("On Junction")
        if junction_shape:
            matrix = detect_surrounding_cars_outside_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )
            for key, value in matrix.items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")
    
    # 2. City junction ahead
    elif is_junction_ahead(ego_waypoint, distance_to_junc) and not highway_junction and junction.id != 459:
        street_type = "Junction ahead"
        junction = get_junction_ahead(ego_waypoint, distance_to_junc)
        junction_shape, lanes_all_junction, junction_roads_junction, yaw, = getJunctionShape(
            ego_vehicle, ego_waypoint, junction, road_lane_ids, direction_angle
        )

        if junction_shape != "Error":
            print("Junction ahead")
            matrix = detect_ego_before_junction(
                deepcopy(junction_shape),
                junction_roads_junction,
                lanes_all_junction,
                ego_waypoint.lane_id,
                ego_waypoint,
                distance_to_junc,
            )
            matrix = detect_surrounding_cars_outside_junction(
                matrix,
                junction_roads_junction,
                lanes_all_junction,
                ego_vehicle,
                world,
                distance_to_junc,
                junction
            )
            matrix = detect_cars_inside_junction(
                matrix, ego_vehicle, junction, yaw, world
            )

            for key, value in dict(matrix).items():
                print(key, value)
        
        else:
            print("Error in recognizing junction shape")



    # 3. Highway entry/exit in front or behind
    elif (is_junction_ahead(ego_waypoint, 80) or (is_junction_behind(ego_waypoint, 40) and street_type == "Highway with entry/exit")) and not on_entry and is_highway_junction and ego_on_highway and junction_id_skip != junction.id:
        street_type = "Highway with entry/exit"
        print("Highway JUNCTION")
        if not same_junction and is_junction_ahead(ego_waypoint, 80):
            print("selbst erkennnen")
            same_junction = True
            junction = get_junction_ahead(ego_waypoint, 80)

            wps = junction.get_waypoints(carla.LaneType().Driving)
            closest_waypoint = get_clostest_starting_waypoint(wps, ego_location)
            wps = prepare_waypoints(closest_waypoint, wps)
            for group in wps:
                for wp in group:
                    print("-------------------")
                    print(wp[0], wp[1])
            highway_shape = get_highway_shape(wps, junction)

            if not highway_shape is None:
                print(highway_shape[0], highway_shape[1])
        if (highway_shape is None) and (not junction_old is None):
            print("alte junction")
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old

        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)            
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            if not highway_shape is None:
                matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape, wps, matrix, ego_waypoint, junction, cars_on_entryExit) 
                if not junction_old is None:
                    matrix = update_matrix(world_map, ego_vehicle, ego_location, highway_shape_old, wps_old, matrix, ego_waypoint, junction_old, cars_on_entryExit) 
                
        for key, value in matrix.items():
            print(value)
        print("----------------------------------------------------------------------------------------------------------------------")


    # 4. Highway entry / exit
    elif is_junction_ahead(ego_waypoint, 20) and not ego_on_highway and is_highway_junction and street_type != "Highway with entry/exit":
        print(junction.id)
        print("On highway entry")
        street_type = "On highway entry"
        on_entry = True
        junction = get_junction_ahead(ego_waypoint, 20)
        junction_id_skip = junction.id

        wps = junction.get_waypoints(carla.LaneType().Driving)
        closest_waypoint, middle_location = get_waypoint_on_highway_junction(ego_waypoint, ego_location, wps, world_map)
        wps = prepare_waypoints(closest_waypoint, wps)
        highway_shape = get_highway_shape(wps, junction)

        on_entry = True
        if (highway_shape is None) and (not junction_old is None):
            same_junction = False
            junction = junction_old
            wps = wps_old
            highway_shape = highway_shape_old


        matrix = create_city_matrix(middle_location, road_lane_ids, world_map, ghost=True)
        if matrix:
            matrix, cars_on_entryExit = detect_surronding_cars(
                middle_location, ego_vehicle, matrix, road_lane_ids, world, radius, True, highway_shape, ghost=True
            )

            if not highway_shape is None:
                matrix  = update_matrix(world_map, ego_vehicle, middle_location, highway_shape, wps, matrix, closest_waypoint, junction, cars_on_entryExit, True, on_entry) 
                if not junction_old is None:
                    matrix = update_matrix(world_map, ego_vehicle, middle_location, highway_shape_old, wps_old, matrix, closest_waypoint, junction_old, cars_on_entryExit, True, on_entry) 
                
        for key, value in matrix.items():
            print(value)
        print("----------")

    # 5. Normal Road
    else:
        on_junction = False
        junction_old = None
        on_entry = False
        print("normal road")
        same_junction = False
        if ego_on_highway:
            street_type = "On highway"
        else:
            street_type = "Non highway street"
        matrix = create_city_matrix(ego_location, road_lane_ids, world_map)
        if matrix:
            matrix, _ = detect_surronding_cars(
                ego_location, ego_vehicle, matrix, road_lane_ids, world, radius, ego_on_highway, highway_shape
            )
            for key, value in matrix.items():
                print(value)

    

    steering_angle = get_steering_angle(ego_vehicle)
    speed = get_speed(ego_vehicle)
    print("============================================================")
    if matrix:
        row_data = get_row(matrix)
        row_data["speed"] = speed
        row_data["steering_angle"] = steering_angle
        row_data["street_type"] = street_type
        df = df.append(row_data, ignore_index=True)

    # clock.tick_busy_loop(60)
    time.sleep(0.3)
    world.tick()


    time.sleep(1)
    world.tick()
    
"""
except Error:
    pass

finally:
    df.to_csv("x_hours.csv")
    vehicles_list = [ego_vehicle]
    print("\ndestroying %d vehicles" % len(vehicles_list))
    client.apply_batch([carla.command.DestroyActor(x) for x in vehicles_list])

    time.sleep(0.5)
"""

----------------------------------
Highway JUNCTION
-------------------
Waypoint(Transform(Location(x=-36.546227, y=-229.551895, z=0.000000), Rotation(pitch=360.000000, yaw=126.499069, roll=0.000000))) end
-------------------
Waypoint(Transform(Location(x=-39.359764, y=-231.633713, z=0.000000), Rotation(pitch=360.000000, yaw=126.499069, roll=0.000000))) end
-------------------
Waypoint(Transform(Location(x=-13.407686, y=-222.575851, z=0.000000), Rotation(pitch=360.000000, yaw=90.797997, roll=0.000000))) end
-------------------
Waypoint(Transform(Location(x=-9.908026, y=-222.527115, z=0.000000), Rotation(pitch=360.000000, yaw=90.797997, roll=0.000000))) end
-------------------
Waypoint(Transform(Location(x=-6.408365, y=-222.478363, z=0.000000), Rotation(pitch=360.000000, yaw=90.797997, roll=0.000000))) end
-------------------
Waypoint(Transform(Location(x=-16.907347, y=-222.624603, z=0.000000), Rotation(pitch=360.000000, yaw=90.797997, roll=0.000000))) end
-------------------
Waypoint(T

KeyboardInterrupt: 

In [ ]:
# ToDo
 
# NEU
# -------------------------------------------------------------------------------------
 # 1) Simplify everything   
 # 2) Überlappende Junctions
 # 3) config schreiben
 
# JP 
#--------------------------------------------------------------------------------------
 # 1) ego_vehicle Dual entry/exit (JP)
 # 2) Doc strings (später)
 # 3) Die ekligen junctions: Tankstelle und Highway Ampel (JP)
 # -----------------------------------------------------------------------------------


# ICH
#------------------------------------------------------------------------------------
 # 1) exitst/entry es mischt sich eine zweispurige straße hinein. Problematisch wenn auto selber drauffährt
